# Preprocess VNP46A1 Nighttime Radiance Data

This notebook provides a workflow to preprocess NASA VNP46A1 nighttime radiance data. The workflow reads HDF5 (.h5) files into arrays, masks pixels for clouds and sensor problems, and exports the preprocessed arrays to GeoTiff files. 

# Notes and References

Links:

* https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/VNP46A1/
* https://viirsland.gsfc.nasa.gov/PDF/VIIRS_BlackMarble_UserGuide.pdf

Download HDF5 files taken care of by `download_laads_order.py`

File naming convention:

VNP46A1.AYYYYDDD.hXXvYY.CCC.YYYYDDDHHMMSS.h5

* VNP46A1 = Short-name
* AYYYYDDD = Acquisition Year and Day of Year
* hXXvYY = Tile Identifier (horizontalXXverticalYY)
* CCC = Collection Version
* YYYYDDDHHMMSS = Production Date – Year, Day, Hour, Minute, Second
* h5 = Data Format (HDF5)

Bands of interest (table from pages 12-13 of handbook)

| Scientific Dataset          | Units             | Description            | Bit Types               | Fill Value | Valid Range | Scale Factor | Offset |
|:-----------------------------|:-------------------|:------------------------|:-------------------------|:------------|:-------------|:--------------|:--------|
| DNB_At_Sensor_Radiance_500m | nW_per_cm2_per_sr | At-sensor DNB radiance | 16-bit unsigned integer | 65535      | 0 - 65534   | 0.1          | 0.0    |
| QF_Cloud_Mask               | Unitless          | Cloud mask status      | 16-bit unsigned integer | 65535      | 0 - 65534   | N/A          | N/A    |
| QF_DNB                      | Unitless          | DNB_quality flag       | 16-bit unsigned integer | 65535      | 0 - 65534   | N/A          | N/A    |
| UTC_Time                    | Decimal hours     | UTC Time               | 32-bit floating point   | -999.9     | 0 - 24      | 1.0          | 0.0    |

How to get Georeferencing info not using GDAL? extract bounds from file using netCDF4 package?

NASA Scripts:

https://git.earthdata.nasa.gov/projects/LPDUR/repos/nasa-viirs/browse/scripts

https://git.earthdata.nasa.gov/projects/LPDUR/repos/nasa-viirs/browse/scripts/VIIRS_HDF5toGeoTIFF.py

Workflow:

* Mask stack for no data
* Mask stack for QA (cloud, snow, shadow, etc.)
* Scale stack
* Fill stack
    * Have different fill values for different layers?
    * And/or have fill values that are of the same `dtype` as the original layer?

Masking Criteria:

* mask where DNB_At_Sensor_Radiance_500m == 65535
* mask where DNB_At_Sensor_Radiance_500m > 65534
* mask where DNB_At_Sensor_Radiance_500m < 0
* mask where QF_Cloud_Mask == 2 (Probably Cloudy)
* mask where QF_Cloud_Mask == 3 (Confident Cloudy)
* mask where QF_DNB != 0 (0 = no problems, any other number means some kind of issue)

# Environment Setup

In [1]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import packages
import os
import re
import glob
import warnings
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.transform import from_origin
import radiance as rd

<IPython.core.display.Javascript object>

In [3]:
# Set options
# sns.set(font_scale=1.5, style="whitegrid")
warnings.simplefilter("ignore")

<IPython.core.display.Javascript object>

In [4]:
# Set working directory
os.chdir("..")
print(f"Working directory: {os.getcwd()}")

Working directory: C:\PSU\08-covid19-remote-sensing-fusion\00-git-repos\nighttime-radiance


<IPython.core.display.Javascript object>

In [5]:
def extract_qa_bits(qa_band, start_bit, end_bit):
    """Extracts the QA bitmask values for a specified
    bitmask (starting and ending bit).

    Parameters
    ----------
    qa_band : numpy array
        Array containing the raw QA values (base-2) for all bitmasks.

    start_bit : int
        First bit in the bitmask.

    end_bit : int
        Last bit in the bitmask.

    Returns
    -------
    qa_values : numpy array
        Array containing the extracted QA values (base-10) for the bitmask.
        
    Example
    -------
        >>>
        >>>
        >>>
        >>>
    """
    # Initialize QA bit string/pattern to check QA band against
    qa_bits = 0

    # Add each specified QA bit flag value/string/pattern
    #  to the QA bits to check/extract
    for bit in range(start_bit, end_bit + 1):
        qa_bits += bit ** 2

    # Check QA band against specified QA bits to see what
    #  QA flag values are set
    qa_flags_set = qa_band & qa_bits

    # Get base-10 value that matches bitmask documentation
    #  (0 or 1 for single bit,  0-3 or 0-N for multiple bits)
    qa_values = qa_flags_set >> start_bit

    return qa_values

<IPython.core.display.Javascript object>

In [6]:
def create_transform_vnp46a1(hdf5):
    """Creates a geographic transform for a VNP46A1 HDF5 file, 
    based on longitude bounds, latitude bounds, and cell size.
    
    Parameters
    ----------
    hdf5 : str
        Path to an existsing VNP46A1 HDF5 file.
    
    Returns
    -------
    transform : affine.Affine object
    
    Example
    -------
        >>>
        >>>
        >>>
        >>>
    """
    # Extract bounding box from top-level dataset
    with rio.open(hdf5) as dataset:
        longitude_min = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_WestBoundingCoord"]
        )
        longitude_max = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_EastBoundingCoord"]
        )
        latitude_min = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_SouthBoundingCoord"]
        )
        latitude_max = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_NorthBoundingCoord"]
        )

        # Extract number of row and columns from first
        #  Science Data Set (subdataset/band)
        with rio.open(dataset.subdatasets[0]) as science_data_set:
            num_rows, num_columns = (
                science_data_set.meta.get("height"),
                science_data_set.meta.get("width"),
            )

    # Define transform (top-left corner, cell size)
    transform = from_origin(
        longitude_min,
        latitude_max,
        (longitude_max - longitude_min) / num_columns,
        (latitude_max - latitude_min) / num_rows,
    )

    return transform

<IPython.core.display.Javascript object>

# Data Acquisition and Preprocessing

Workflow:
* Download HDF files
* Load HDF files
* Read radiance band into array
* Read cloud mask data into array 
* Read other QA data into array
* Create stack of bands to use
* Clip stack
* Mask for DNB band quality
* Mask for clouds
* Mask radiance based on QA/cloud mask flags
* Set no data/masked to 0
* Set data outside bounds to 0
* Store in dictionary based on year/month/day/time
* Get metadata
* Export radiance to GeoTiff



In [7]:
# Get HDF files into list
hdf_files = glob.glob(
    os.path.join("02-raw-data", "hdf", "south-korea", "*.h5")
)
print(f"There are {len(hdf_files)} HDF files to preprocess.")

There are 10 HDF files to preprocess.


<IPython.core.display.Javascript object>

In [8]:
# def extract_science_data_sets(hdf5_path):
#     """Extracts the DNB_At_Sensor_Radiance_500m, QF_Cloud_Mask, and QF_DNB
#     science data sets (SDS) from a NASA VNP46A1 HDF5 file.

#     Parameters
#     ----------
#     hdf5_path : str
#         Path to the VNP46A1 HDF5 (.h5) file.

#     Returns
#     -------
#     tuple

#         dnb_at_sensor_radiance : numpy array
#            Array containing at-sensor radiance values.

#         qf_dnb : numpy array
#             Array containing quality flag values for the at-sensor radiance
#             (sensor-related effects).

#         qf_cloud_mask
#             Array containing the quality flag values for environment-related
#             effects (clouds, shadows, land/water).

#     Example
#     -------
#         >>>
#         >>>
#         >>>
#     """
#     # Open top-level dataset
#     with rio.open(hdf5_path) as dataset:
#         # Loop through subdatasets (Science Data Sets)
#         for science_data_set in dataset.subdatasets:
#             # Extract DNB_At_Sensor_Radiance_500m, QF_Cloud_Mask, QF_DNB
#             if re.search("DNB_At_Sensor_Radiance_500m$", science_data_set):
#                 with rio.open(science_data_set) as source:
#                     dnb_at_sensor_radiance = source.read(1)
#             if re.search("QF_Cloud_Mask$", science_data_set):
#                 with rio.open(science_data_set) as source:
#                     qf_cloud_mask = source.read(1)
#             if re.search("QF_DNB$", science_data_set):
#                 with rio.open(science_data_set) as source:
#                     qf_dnb = source.read(1)

#     return dnb_at_sensor_radiance, qf

<IPython.core.display.Javascript object>

In [9]:
def extract_band_vnp46a1(hdf5_path, band_name):
    """Extracts the specified band (Science Data Set) from a NASA VNP46A1 HDF5
    file.
    
    Available Science Data Sets include:
    
    BrightnessTemperature_M12
    Moon_Illumination_Fraction
    Moon_Phase_Angle
    QF_Cloud_Mask
    QF_DNB
    QF_VIIRS_M10
    QF_VIIRS_M11
    QF_VIIRS_M12
    QF_VIIRS_M13
    QF_VIIRS_M15
    QF_VIIRS_M16
    BrightnessTemperature_M13
    Radiance_M10
    Radiance_M11
    Sensor_Azimuth
    Sensor_Zenith
    Solar_Azimuth
    Solar_Zenith
    UTC_Time
    BrightnessTemperature_M15
    BrightnessTemperature_M16
    DNB_At_Sensor_Radiance_500m
    Glint_Angle
    Granule
    Lunar_Azimuth
    Lunar_Zenith
    
    Parameters
    ----------
    hdf5_path : str
        Path to the VNP46A1 HDF5 (.h5) file. 
        
    band_name : str
        Name of the band (Science Data Set) to be extracted. Must be an exact
        match to an available Science Data Set.     
    
    Returns
    -------
    band : numpy array
        Array containing the data for the specified band (Science Data Set).
    
    Example
    -------
        >>> qf_cloud_mask = extract_band_vnp46a1(
        ...     hdf5='VNP46A1.A2020001.h30v05.001.2020004003738.h5',
        ...     band='QF_Cloud_Mask'
        ... )
        >>> type(qf_cloud_mask)
        numpy.ndarray
    """
    # Raise error for invalid band name
    band_names = [
        "BrightnessTemperature_M12",
        "Moon_Illumination_Fraction",
        "Moon_Phase_Angle",
        "QF_Cloud_Mask",
        "QF_DNB",
        "QF_VIIRS_M10",
        "QF_VIIRS_M11",
        "QF_VIIRS_M12",
        "QF_VIIRS_M13",
        "QF_VIIRS_M15",
        "QF_VIIRS_M16",
        "BrightnessTemperature_M13",
        "Radiance_M10",
        "Radiance_M11",
        "Sensor_Azimuth",
        "Sensor_Zenith",
        "Solar_Azimuth",
        "Solar_Zenith",
        "UTC_Time",
        "BrightnessTemperature_M15",
        "BrightnessTemperature_M16",
        "DNB_At_Sensor_Radiance_500m",
        "Glint_Angle",
        "Granule",
        "Lunar_Azimuth",
        "Lunar_Zenith",
    ]
    if band_name not in band_names:
        raise ValueError(
            f"Invalid band name. Must be one of the following: {band_names}"
        )

    # Open top-level dataset, loop through Science Data Sets (subdatasets),
    #  and extract specified band
    with rio.open(hdf5_path) as dataset:
        for science_data_set in dataset.subdatasets:
            if re.search(f"{band_name}$", science_data_set):
                with rio.open(science_data_set) as src:
                    band = src.read(1)

    return band

<IPython.core.display.Javascript object>

In [ ]:
extract_band_vnp46a1(hdf5_path=hdf_files[1], band_name="QF_Cloud_Mask").shape

In [11]:
# # Loop through each HDF file
# for hdf in [hdf_files[0]]:
#     # Open top-level dataset
#     with rio.open(hdf) as dataset:
#         # Loop through subdatasets (Science Data Sets)
#         for science_data_set in dataset.subdatasets:
#             # Get name of Science Data Set (SDS)
#             science_data_set_name = science_data_set.split(os.sep)[-1].split(
#                 "/"
#             )[-1]
#             print(science_data_set_name)

<IPython.core.display.Javascript object>

In [13]:
def preprocess_vnp46a1(hdf5_path, output_folder):
    """Preprocessed a NASA VNP46A1 HDF5 (.h5 file)
    
    Preprocessing steps include masking data for fill values, clouds, and 
    sensor problems, filling masked values, and exporting data to a GeoTiff.
    
    Parameters
    ----------
    hdf5_path : str
        Path to the VNP46A1 HDF5 (.h5) file to be preprocessed. 
    
    output_folder : str
        Path to the folder where the preprocessed file will be exported to.
    
    Returns
    -------
    message : str
        Indication of preprocessing completion status (success or failure).
    
    Example
    -------
        >>>
        >>>
        >>>
        >>>
    """
    # Preprocess VNP46A1 HDF5 file
    print(f"Started preprocessing: {os.path.basename(hdf5_path)}")
    try:
        print("Extracting bands...")
        # Extract DNB_At_Sensor_Radiance_500m, QF_Cloud_Mask, QF_DNB
        dnb_at_sensor_radiance = extract_band_vnp46a1(
            hdf5_path=hdf, band_name="DNB_At_Sensor_Radiance_500m"
        )
        qf_cloud_mask = extract_band_vnp46a1(
            hdf5_path=hdf, band_name="QF_Cloud_Mask"
        )
        qf_dnb = extract_band_vnp46a1(hdf5_path=hdf, band_name="QF_DNB")

        print("Applying scale factor...")
        # Apply scale factor to radiance values
        dnb_at_sensor_radiance_scaled = (
            dnb_at_sensor_radiance.astype("float") * 0.1
        )

        print("Masking for fill values...")
        # Mask radiance for fill value (DNB_At_Sensor_Radiance_500m == 65535)
        masked_for_fill_value = ma.masked_where(
            dnb_at_sensor_radiance_scaled == 6553.5,
            dnb_at_sensor_radiance_scaled,
            copy=True,
        )

        print("Masking for clouds...")
        # Extract QF_Cloud_Mask bits 6-7 (Cloud Detection Results &
        #  Confidence Indicator)
        cloud_detection_bitmask = extract_qa_bits(
            qa_band=qf_cloud_mask, start_bit=6, end_bit=7
        )

        # Mask radiance for 'probably cloudy' (cloud_detection_bitmask == 2)
        masked_for_probably_cloudy = ma.masked_where(
            cloud_detection_bitmask == 2, masked_for_fill_value, copy=True
        )

        # Mask radiance for 'confident cloudy' (cloud_detection_bitmask == 3)
        masked_for_confident_cloudy = ma.masked_where(
            cloud_detection_bitmask == 3, masked_for_probably_cloudy, copy=True
        )

        print("Masking for sensor problems...")
        # Mask radiance for sensor problems (QF_DNB != 0)
        #  (0 = no problems, any number > 0 means some kind of issue)
        masked_for_sensor_problems = ma.masked_where(
            qf_dnb > 0, masked_for_confident_cloudy, copy=True
        )

        print("Filling masked values...")
        # Set fill value to np.nan and fill masked values
        ma.set_fill_value(masked_for_sensor_problems, np.nan)
        filled_data = masked_for_sensor_problems.filled()

        print("Creating metadata...")
        # Create metadata (for export)
        metadata = rd.create_metadata(
            array=filled_data,
            transform=create_transform_vnp46a1(hdf),
            driver="GTiff",
            nodata=np.nan,
            count=1,
            crs="epsg:4326",
        )

        print("Exporting to GeoTiff...")
        # Export masked array to GeoTiff (no data set to np.nan in export)
        rd.export_array(
            array=filled_data,
            output_path=os.path.join(
                output_folder,
                f"{os.path.basename(hdf)[:-3].lower().replace('.', '-')}.tif",
            ),
            metadata=metadata,
        )
    except Exception as error:
        message = print(f"Preprocessing failed: {error}")
    else:
        message = print(
            f"Completed preprocessing: {os.path.basename(hdf5_path)}\n"
        )

    return message

<IPython.core.display.Javascript object>

* Extract bands
* Mask for fill values
* Mask for clouds
* Mask for sensor problems
* Fill masked values
* Create transform
* Create metadata
* Export array to GeoTiff

In [14]:
# Preprocess each HDF file
for hdf in hdf_files:
    preprocess_vnp46a1(
        hdf5_path=hdf,
        output_folder=os.path.join(
            "03-processed-data", "raster", "south-korea", "vnp46a1-grid"
        ),
    )

Started preprocessing: VNP46A1.A2020001.h30v05.001.2020004003738.h5
Extracting bands...
Applying scale factor...
Masking for fill values...
Masking for clouds...
Masking for sensor problems...
Filling masked values...
Creating metadata...
Exporting to GeoTiff...
Exported: vnp46a1-a2020001-h30v05-001-2020004003738.tif
Completed preprocessing: VNP46A1.A2020001.h30v05.001.2020004003738.h5

Started preprocessing: VNP46A1.A2020001.h31v05.001.2020004003841.h5
Extracting bands...
Applying scale factor...
Masking for fill values...
Masking for clouds...
Masking for sensor problems...
Filling masked values...
Creating metadata...
Exporting to GeoTiff...
Exported: vnp46a1-a2020001-h31v05-001-2020004003841.tif
Completed preprocessing: VNP46A1.A2020001.h31v05.001.2020004003841.h5

Started preprocessing: VNP46A1.A2020002.h30v05.001.2020004085319.h5
Extracting bands...
Applying scale factor...
Masking for fill values...
Masking for clouds...
Masking for sensor problems...
Filling masked values...
Cr

<IPython.core.display.Javascript object>

In [ ]:
# Loop through each HDF file
for hdf in hdf_files:
    # Extract DNB_At_Sensor_Radiance_500m, QF_Cloud_Mask, QF_DNB
    dnb_at_sensor_radiance = extract_band_vnp46a1(
        hdf5_path=hdf, band_name="DNB_At_Sensor_Radiance_500m"
    )
    qf_cloud_mask = extract_band_vnp46a1(
        hdf5_path=hdf, band_name="QF_Cloud_Mask"
    )
    qf_dnb = extract_band_vnp46a1(hdf5_path=hdf, band_name="QF_DNB")

    # Apply scale factor to radiance values
    dnb_at_sensor_radiance_scaled = (
        dnb_at_sensor_radiance.astype("float") * 0.1
    )

    # Mask radiance for fill value (DNB_At_Sensor_Radiance_500m == 65535)
    masked_for_fill_value = ma.masked_where(
        dnb_at_sensor_radiance_scaled == 6553.5,
        dnb_at_sensor_radiance_scaled,
        copy=True,
    )

    # Extract QF_Cloud_Mask bits 6-7 (Cloud Detection Results &
    #  Confidence Indicator)
    cloud_detection_bitmask = extract_qa_bits(
        qa_band=qf_cloud_mask, start_bit=6, end_bit=7
    )

    # Mask radiance for 'probably cloudy' (cloud_detection_bitmask == 2)
    masked_for_probably_cloudy = ma.masked_where(
        cloud_detection_bitmask == 2, masked_for_fill_value, copy=True
    )

    # Mask radiance for 'confident cloudy' (cloud_detection_bitmask == 3)
    masked_for_confident_cloudy = ma.masked_where(
        cloud_detection_bitmask == 3, masked_for_probably_cloudy, copy=True
    )

    # Mask radiance for sensor problems (QF_DNB != 0)
    #  (0 = no problems, any number > 0 means some kind of issue)
    masked_for_sensor_problems = ma.masked_where(
        qf_dnb > 0, masked_for_confident_cloudy, copy=True
    )

    # Set fill value to np.nan and fill masked values
    ma.set_fill_value(masked_for_sensor_problems, np.nan)
    filled_data = masked_for_sensor_problems.filled()

    print(f"Max scaled value: {np.nanmax(filled_data)}")
    print(f"Mean scaled value: {np.nanmean(filled_data)}")
    print(f"Median scaled value: {np.nanmedian(filled_data)}")
    print(
        f"Number of masked pixels: {np.count_nonzero(np.isnan(filled_data))}\n"
    )

    # Create metadata (for export)
    metadata = rd.create_metadata(
        array=filled_data,
        transform=create_transform_vnp46a1(hdf),
        driver="GTiff",
        nodata=np.nan,
        count=1,
        crs="epsg:4326",
    )

    # Export masked array to GeoTiff (no data set to np.nan in export)
    rd.export_array(
        array=filled_data,
        output_path=os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "vnp46a1-grid",
            f"{os.path.basename(hdf)[:-3].lower().replace('.', '-')}.tif",
        ),
        metadata=metadata,
    )

In [ ]:
# Loop through each HDF file
for hdf in hdf_files:
    # Open top-level dataset
    with rio.open(hdf) as dataset:
        # Loop through subdatasets (Science Data Sets)
        for science_data_set in dataset.subdatasets:
            #             # Get name of Science Data Set (SDS)
            #             science_data_set_name = science_data_set.split(os.sep)[-1].split(
            #                 "/"
            #             )[-1]
            #             print(science_data_set)
            # Extract Science Data Sets of interest
            #  (DNB_At_Sensor_Radiance_500m, QF_Cloud_Mask, QF_DNB)
            if re.search("DNB_At_Sensor_Radiance_500m$", science_data_set):
                with rio.open(science_data_set) as source:
                    dnb_at_sensor_radiance = source.read(1)
            if re.search("QF_Cloud_Mask$", science_data_set):
                with rio.open(science_data_set) as source:
                    qf_cloud_mask = source.read(1)
            if re.search("QF_DNB$", science_data_set):
                with rio.open(science_data_set) as source:
                    qf_dnb = source.read(1)

    # Convert at-sensor radiance array to type float
    dnb_at_sensor_radiance_float = dnb_at_sensor_radiance.astype("float")

    # Apply scale factor to radiance values
    dnb_at_sensor_radiance_scaled = dnb_at_sensor_radiance_float * 0.1

    # Mask radiance for fill value (DNB_At_Sensor_Radiance_500m == 65535)
    masked_for_fill_value = ma.masked_where(
        dnb_at_sensor_radiance_scaled == 6553.5,
        dnb_at_sensor_radiance_scaled,
        copy=True,
    )

    # Mask radiance for outside valid range (DNB_At_Sensor_Radiance_500m > 65534)
    # Mask radiance for outside valid range (DNB_At_Sensor_Radiance_500m < 0)

    # Extract QF_Cloud_Mask bits 6-7 (Cloud Detection Results & Confidence Indicator)
    cloud_detection_bitmask = extract_qa_bits(
        qa_band=qf_cloud_mask, start_bit=6, end_bit=7
    )

    # Mask radiance for 'probably cloudy' (cloud_detection_bitmask == 2)
    masked_for_probably_cloudy = ma.masked_where(
        cloud_detection_bitmask == 2, masked_for_fill_value, copy=True
    )

    # Mask radiance for 'confident cloudy' (cloud_detection_bitmask == 3)
    masked_for_confident_cloudy = ma.masked_where(
        cloud_detection_bitmask == 3, masked_for_probably_cloudy, copy=True
    )

    # Mask radiance for sensor problems (QF_DNB != 0)
    #  (0 = no problems, any number > 0 means some kind of issue)
    masked_for_sensor_problems = ma.masked_where(
        qf_dnb > 0, masked_for_confident_cloudy, copy=True
    )

    #     print(masked_for_sensor_problems)

    # Extract QF_Cloud_Mask bit 9 (Cirrus detection)
    #     cirrus_detection_bitmask = extract_qa_bits(
    #         qa_band=qf_cloud_mask, start_bit=9, end_bit=9
    #     )

    # Mask radiance for Cirrus detection (cirrus_detection_bitmask == 1)
    #     masked_for_cirrus = ma.masked_where(
    #         cirrus_detection_bitmask == 1, masked_for_sensor_problems, copy=True
    #     )

    # Fill masked values with np.nan or 0 - NEED TO FIX THIS VALUE OR DTYPE
    # Set fill value to np.nan
    #     ma.set_fill_value(masked_for_sensor_problems, 0)
    #     ma.set_fill_value(masked_for_sensor_problems, -99990.0)
    ma.set_fill_value(masked_for_sensor_problems, np.nan)

    # Fill masked values with np.nan
    filled_data = masked_for_sensor_problems.filled()
    #     filled_data = masked_for_cirrus.filled()

    # Count the number of NaN values
    #     np.count_nonzero(np.isnan(filled_data))
    # np.nan is np.NaN is np.NAN ==> True

    #     print(f"Max value after masking")
    # Apply scale factor to data
    #     scaled_array = apply_scale_factor(filled_array, scale_factor=scale_factor)
    # Apply scale factor to data
    #     scaled_data = filled_data * 0.1

    print(f"Max scaled value: {np.nanmax(filled_data)}")
    print(f"Mean scaled value: {np.nanmean(filled_data)}")
    print(f"Median scaled value: {np.nanmedian(filled_data)}")
    print(
        f"Number of masked pixels: {np.count_nonzero(np.isnan(filled_data))}\n"
    )

    # Set file export name
    export_name = f"{os.path.basename(hdf)[:-3].lower().replace('.', '-')}.tif"

    # Create transform (for export)
    transform = create_transform_vnp46a1(hdf)

    # Create metadata (for export)
    metadata = rd.create_metadata(
        array=filled_data,
        transform=transform,
        driver="GTiff",
        #         nodata=-9999.0,  # NEED TO CHANGE THIS
        #         nodata=0,  # NEED TO CHANGE THIS
        nodata=np.nan,  # NEED TO CHANGE THIS
        count=1,
        crs="epsg:4326",
    )

    # Export masked array to GeoTiff (no data set to np.nan in export)
#     rd.export_array(
#         array=scaled_data,
#         output_path=os.path.join(
#             "03-processed-data",
#             "raster",
#             "south-korea",
#             "vnp46a1-grid",
#             export_name,
#         ),
#         metadata=metadata,
#     )

In [ ]:
# Define list of bands used in preprocessing
required_bands = [
    "DNB_At_Sensor_Radiance_500m",
    "QF_Cloud_Mask",
    "QF_DNB",
    "UTC_Time",
]

# Loop through each HDF file
for hdf in hdf_files:
    # Open top-level dataset
    with rio.open(hdf) as dataset:
        # Loop through subdatasets (Science Data Sets)
        for science_data_set in dataset.subdatasets:
            #             # Get name of Science Data Set (SDS)
            #             science_data_set_name = science_data_set.split(os.sep)[-1].split(
            #                 "/"
            #             )[-1]
            #             print(science_data_set)
            # Extract Science Data Sets of interest
            #  (DNB_At_Sensor_Radiance_500m, QF_Cloud_Mask, QF_DNB)
            if re.search("DNB_At_Sensor_Radiance_500m$", science_data_set):
                with rio.open(science_data_set) as source:
                    dnb_at_sensor_radiance = source.read(1)
            if re.search("QF_Cloud_Mask$", science_data_set):
                with rio.open(science_data_set) as source:
                    qf_cloud_mask = source.read(1)
            if re.search("QF_DNB$", science_data_set):
                with rio.open(science_data_set) as source:
                    qf_dnb = source.read(1)

    # Apply scale factor to radiance values data
    dnb_at_sensor_radiance_scaled = dnb_at_sensor_radiance * 0.1
                    
    # Mask radiance for fill value (DNB_At_Sensor_Radiance_500m == 65535)
    masked_for_fill_value = ma.masked_where(
        dnb_at_sensor_radiance == 65535, dnb_at_sensor_radiance, copy=True
    ).astype("int32")

    # Mask radiance for outside valid range (DNB_At_Sensor_Radiance_500m > 65534)
    # Mask radiance for outside valid range (DNB_At_Sensor_Radiance_500m < 0)

    # Extract QF_Cloud_Mask bits 6-7 (Cloud Detection Results & Confidence Indicator)
    cloud_detection_bitmask = extract_qa_bits(
        qa_band=qf_cloud_mask, start_bit=6, end_bit=7
    )

    # Mask radiance for 'probably cloudy' (cloud_detection_bitmask == 2)
    masked_for_probably_cloudy = ma.masked_where(
        cloud_detection_bitmask == 2, masked_for_fill_value, copy=True
    )

    # Mask radiance for 'confident cloudy' (cloud_detection_bitmask == 3)
    masked_for_confident_cloudy = ma.masked_where(
        cloud_detection_bitmask == 3, masked_for_probably_cloudy, copy=True
    )

    # Mask radiance for sensor problems (QF_DNB != 0)
    #  (0 = no problems, any number > 0 means some kind of issue)
    masked_for_sensor_problems = ma.masked_where(
        qf_dnb > 0, masked_for_confident_cloudy, copy=True
    )

    print(masked_for_sensor_problems)

    # Extract QF_Cloud_Mask bit 9 (Cirrus detection)
    #     cirrus_detection_bitmask = extract_qa_bits(
    #         qa_band=qf_cloud_mask, start_bit=9, end_bit=9
    #     )

    # Mask radiance for Cirrus detection (cirrus_detection_bitmask == 1)
    #     masked_for_cirrus = ma.masked_where(
    #         cirrus_detection_bitmask == 1, masked_for_sensor_problems, copy=True
    #     )

    # Fill masked values with np.nan or 0 - NEED TO FIX THIS VALUE OR DTYPE
    # Change fill value to NaN
    #     ma.set_fill_value(masked_for_sensor_problems, 0)
    ma.set_fill_value(masked_for_sensor_problems, -99990.0)
    #     ma.set_fill_value(masked_for_sensor_problems, np.nan)

    # Fill masked values with NaN
    filled_data = masked_for_sensor_problems.filled()
    #     filled_data = masked_for_cirrus.filled()

    #     print(f"Max value after masking")
    # Apply scale factor to data
    #     scaled_array = apply_scale_factor(filled_array, scale_factor=scale_factor)
    # Apply scale factor to data
    scaled_data = filled_data * 0.1

    print(f"Max scaled value: {scaled_data.max()}")
    print(f"Mean scaled value: {np.mean(scaled_data)}")
    print(f"Median scaled value: {np.median(scaled_data)}\n")

    # Set file export name
    export_name = f"{os.path.basename(hdf)[:-3].lower().replace('.', '-')}.tif"

    # Create transform (for export)
    transform = create_transform_vnp46a1(hdf)

    # Create metadata (for export)
    metadata = rd.create_metadata(
        array=scaled_data,
        transform=transform,
        driver="GTiff",
        nodata=-9999.0,  # NEED TO CHANGE THIS
        #         nodata=0,  # NEED TO CHANGE THIS
        #         nodata=np.nan, # NEED TO CHANGE THIS
        count=1,
        crs="epsg:4326",
    )

    # Export masked array to GeoTiff (no data set to np.nan in export)
    rd.export_array(
        array=scaled_data,
        output_path=os.path.join(
            "03-processed-data",
            "raster",
            "south-korea",
            "vnp46a1-grid",
            export_name,
        ),
        metadata=metadata,
    )

In [ ]:
transform

In [ ]:
np.concatenate((scaled_data, scaled_data), axis=1).shape

In [ ]:
transform = create_transform_vnp46a1(hdf_files[0])

In [ ]:
transform

In [ ]:
transform = create_transform_vnp46a1(hdf_files[1])
transform

In [ ]:
type(transform)

In [ ]:
export_name

In [ ]:
max_value_index = np.unravel_index(
    np.argmax(masked_for_sensor_problems, axis=None),
    masked_for_sensor_problems.shape,
)

max_value_index

# a[ind]
# 15

In [ ]:
ep.plot_bands(masked_for_sensor_problems)
# np.median(masked_for_sensor_problems)
# np.argmax(masked_for_sensor_problems, axis=1)

In [ ]:
# Read in full geotiff
# Crop to south korea boundary (get cropped image and metadata) es.crop_image
# Export cropped array to geotiff

In [ ]:
# Extract bitmasks for masking radiance data
day_night_bitmask = extract_qa_bits(
    qa_band=qf_cloud_mask, start_bit=0, end_bit=0
)

land_water_background_bitmask = extract_qa_bits(
    qa_band=qf_cloud_mask, start_bit=1, end_bit=3
)

cloud_mask_quality_bitmask = extract_qa_bits(
    qa_band=qf_cloud_mask, start_bit=4, end_bit=5
)

cloud_detection_bitmask = extract_qa_bits(
    qa_band=qf_cloud_mask, start_bit=6, end_bit=7
)

shadow_detected_bitmask = extract_qa_bits(
    qa_band=qf_cloud_mask, start_bit=8, end_bit=8
)

cirrus_detection_bitmask = extract_qa_bits(
    qa_band=qf_cloud_mask, start_bit=9, end_bit=9
)

snow_ice_surface_bitmask = extract_qa_bits(
    qa_band=qf_cloud_mask, start_bit=10, end_bit=10
)

In [ ]:
day_night_bitmask[max_value_index] # 0 = Night

In [ ]:
land_water_background_bitmask[max_value_index] # 3 = Sea Water

In [ ]:
cloud_mask_quality_bitmask[max_value_index] # 2 = Medium

In [ ]:
cloud_detection_bitmask[max_value_index] # 0 = Confident Clear

In [ ]:
shadow_detected_bitmask[max_value_index] # 0 = No Shadow

In [ ]:
cirrus_detection_bitmask[max_value_index]  # 0 = No Cloud

In [ ]:
snow_ice_surface_bitmask[max_value_index] # 0 = No Snow/Ice

In [ ]:
np.unique(qf_dnb)

In [ ]:
qf_dnb[max_value_index] # 0 = No problems

In [ ]:
dnb_at_sensor_radiance.shape

In [ ]:
dnb_at_sensor_radiance_metadata

In [ ]:
dnb_at_sensor_radiance_profile

In [ ]:
for hdf in [hdf_files[0]]:
    # Open top-level dataset (has no geospatial information)
    with rio.open(hdf) as dataset:
        print(dataset)

In [ ]:
for hdf in [hdf_files[0]]:
    print(hdf["HDFEO)
    # Open top-level dataset (has no geospatial information)
#     with rio.open(hdf) as dataset:
#         print(dataset)

In [ ]:
with rio.open(hdf_files[0]) as dataset:
    print(dataset)
    hdf5_metadata = dataset.meta
    crs = dataset.read_crs()
    print(dataset.read_crs())  # crs is None
    print(type(dataset.tags()))
    print(
        dataset.tags()
    )  # ['NorthBoundingCoord']), EastBoundingCoord, WestBoundingCoord, SouthBoundingCoord
    

In [ ]:
with rio.open(hdf_files[0]) as dataset:
    print(dataset)
    hdf5_metadata = dataset.meta
    crs = dataset.read_crs()
    print(dataset.read_crs())  # crs is None
    print(type(dataset.tags()))
    #     print(
    #         dataset.tags()
    #     )  # ['NorthBoundingCoord']), EastBoundingCoord, WestBoundingCoord, SouthBoundingCoord
    for key in dataset.tags():
        #         if "EastBounding" in key:
        print(f"{key}: {dataset.tags()[key]}\n")

In [ ]:
def create_transform_vnp46a1(hdf5):
    """Creates a geographic transform for a VNP46A1 HDF5 file, 
    based on longitude bounds, latitude bounds, and cell size.
    """
    # Extract bounding coordinates and number of rows/colums from top-level
    with rio.open(hdf5) as dataset:
        longitude_min = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_WestBoundingCoord"]
        )
        longitude_max = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_EastBoundingCoord"]
        )
        latitude_min = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_SouthBoundingCoord"]
        )
        latitude_max = int(
            dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_NorthBoundingCoord"]
        )

        # Extract cell size from first band (science data set)
        with rio.open(dataset.subdatasets[0]) as band:
            num_rows, num_columns = (
                band.meta.get("height"),
                band.meta.get("width"),
            )

    # Define transform (top-left corner, cell size)
    transform = from_origin(
        longitude_min,
        latitude_max,
        (longitude_max - longitude_min) / num_columns,
        (latitude_max - latitude_min) / num_rows,
    )

    return transform

In [ ]:
transform = create_transform_vnp46a1(hdf_files[0])
transform

In [ ]:
with rio.open(hdf_files[0]) as dataset:
    longitude_min = int(
        dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_WestBoundingCoord"]
    )
    longitude_max = int(
        dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_EastBoundingCoord"]
    )
    latitude_min = int(
        dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_SouthBoundingCoord"]
    )
    latitude_max = int(
        dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_NorthBoundingCoord"]
    )

    # Extract cell size from first band (science data set)
    with rio.open(dataset.subdatasets[0]) as band:
        num_rows, num_columns = band.meta.get("height"), band.meta.get("width")
    #         print(band.meta)

In [ ]:
def extract_image_date_vnp46a1(hdf5):
    """Returns the acquisition date of a VNP46A1 HDF5 file.
    
    Parameters
    ----------
    hdf5 : str
        Path to the VNP46A1 HDF5 file.
        
    Returns
    -------
    date : str
        Acquisition date of the image, formatted as 'YYYY-MM-DD'.
    
    Example
    -------
        >>> hdf5_file = "VNP46A1.A2020001.h30v05.001.2020004003738.h5"
        >>> extract_image_date_vnp46a1(hdf5_file)
        '2020-01-01'   
    """
    # Open file and extract date
    with rio.open(hdf5) as dataset:
        date = dataset.tags()["HDFEOS_GRIDS_VNP_Grid_DNB_RangeBeginningDate"]

    return date

In [ ]:
extract_image_date_vnp46a1(hdf_files[0])

* QF_Cloud_Mask
    * Bit 0 - Day/Night 	
        * 0 (Night)
        * 1 (Day)
        
    * Bits 1-3 - Land/Water Background
        * 0 (Land & Desert)
        * 1 (Land no Desert)
        * 2 (Inland Water)
        * 3 (Sea Water)
        * 5 (Coastal)
        
    * Bits 4-5 - Cloud Mask Quality 	
        * 0 (Poor)
        * 1 (Low)
        * 2 (Medium)
        * 3 (High)
    
    * Bits 6-7 - Cloud Detection Results & Confidence Indicator 
        * 0 (Confident Clear)
        * 1 (Probably Clear)
        * 2 (Probably Cloudy)
        * 3 (Confident Cloudy)

    * Bit 8 - Shadow Detected
        * 1 (Yes)
        * 0 (No)

    * Bit 9 - Cirrus Detection (IR) (BTM15 –BTM16) 	
        * 1 (Cloud)
        * 0 (No Cloud)

    * Bit 10 - Snow/Ice Surface
        * 1 (Snow/Ice)
        * 0 (No Snow/Ice)

QF_Cloud_Mask (base-2):

| Bit | Flag Description Key                          | Interpretation                                                                            |
|:-----|:-----------------------------------------------|:-------------------------------------------------------------------------------------------|
| 0   | Day/Night                                     | 0 = Night <br> 1 = Day                                                                         |
| 1-3 | Land/Water Background                         | 000 = Land & Desert <br> 001 = Land no Desert <br> 010 = Inland Water <br> 011 = Sea Water <br> 101 = Coastal |
| 4-5 | Cloud Mask Quality                            | 00 = Poor <br> 01 = Low <br> 10 = Medium <br> 11 = High                                                  |
| 6-7 | Cloud Detection Results & Confidence Indicator | 00 = Confident Clear <br> 01 = Probably Clear <br> 10 = Probably Cloudy <br> 11 = Confident Cloudy     |
| 8   | Shadow Detected                               | 1 = Yes <br> 0 = No                                                                            |
| 9   | Cirrus Detection (IR) (BTM15 –BTM16)          | 1 = Cloud <br> 0 = No Cloud                                                                   |
| 10  | Snow/Ice Surface                              | 1 = Snow/Ice <br> 0 = No Snow/Ice                                                            |

QF_Cloud_Mask (base-10):

| Bit | Flag Description Key                          | Interpretation                                                                            |
|:-----|:-----------------------------------------------|:-------------------------------------------------------------------------------------------|
| 0   | Day/Night                                     | 0 = Night <br> 1 = Day                                                                         |
| 1-3 | Land/Water Background                         | 0 = Land & Desert <br> 1 = Land no Desert <br> 2 = Inland Water <br> 3 = Sea Water <br> 5 = Coastal |
| 4-5 | Cloud Mask Quality                            | 0 = Poor <br> 1 = Low <br> 2 = Medium <br> 3 = High                                                  |
| 6-7 | Cloud Detection Results & Confidence Indicator | 0 = Confident Clear <br> 1 = Probably Clear <br> 2 = Probably Cloudy <br> 3 = Confident Cloudy     |
| 8   | Shadow Detected                               | 1 = Yes <br> 0 = No                                                                            |
| 9   | Cirrus Detection (IR) (BTM15 –BTM16)          | 1 = Cloud <br> 0 = No Cloud                                                                   |
| 10  | Snow/Ice Surface                              | 1 = Snow/Ice <br> 0 = No Snow/Ice                                                            |

QF_DNB:

| SDS Layer | Flag Mask Values and Descriptions|
|:-----------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| QF_DNB    | 1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Substitute_Cal<br>2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Out_of_Range<br>4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Saturation<br>8&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Temp_not_Nominal<br>16&nbsp;&nbsp;&nbsp;&nbsp; = Stray_Light<br>256&nbsp;&nbsp; = Bowtie_Deleted/Range_Bit<br>512&nbsp;&nbsp; = Missing_EV<br>1024 = Cal_Fail<br>2048 = Dead_Detector |

In [ ]:
def extract_qa_bits(qa_band, start_bit, end_bit):
    """Extracts the QA bitmask values for a specified
    bitmask (starting and ending bit).

    Parameters
    ----------
    qa_band : numpy array
        Array containing the raw QA values (base-2) for all bitmasks.

    start_bit : int
        First bit in the bitmask.

    end_bit : int
        Last bit in the bitmask.

    Returns
    -------
    qa_values :  numpy array
        Array containing the extracted QA values (base-10) for the bitmask.
        
    Example
    -------

    """
    # Initialize QA bit string/pattern to check QA band against
    qa_bits = 0

    # Add each specified QA bit flag value/string/pattern
    #  to the QA bits to check/extract
    for bit in range(start_bit, end_bit + 1):
        qa_bits += bit ** 2

    # Check QA band against specified QA bits to see what QA flag values are set
    qa_flags_set = qa_band & qa_bits

    # Get base-10 value that matches bitmask documentation
    #  (0 or 1 for single bit,  0-3 or 0-N for multiple bits)
    qa_values = qa_flags_set >> start_bit

    # Return array of qa_values for specific bits
    return qa_values

In [ ]:
latitude_min

In [ ]:
num_columns

In [ ]:
def stack_science_datasets(hdf5):
    """Stacks all 7 Science Data Sets into a single array.

    Parameters
    ----------
    hdf5 : str
        Path to the HDF5 file.

    """
    # Initialize list to store all science datasets
    science_datasets = []

    # Populate list with all science datasets
    with rio.open(hdf5_file) as dataset:
        for science_dataset in dataset.subdatasets:
            with rio.open(science_dataset) as band:
                science_datasets.append(band.read(1))

    # Stack science datasets into single array            
    science_datasets_stacked = np.stack(science_datasets)
    
    return science_datasets_stacked

# Data Processing

# Data Postprocessing

# Data Visualization

# Data Export

# Scratch Work

In [ ]:
# Define list of bands used in preprocessing
required_bands = [
    "DNB_At_Sensor_Radiance_500m",
    "QF_Cloud_Mask",
    "QF_DNB",
    "UTC_Time",
]

# Loop through each HDF file
for hdf in [hdf_files[0]]:
    # Open top-level dataset (has no geospatial information)
    with rio.open(hdf) as dataset:
        #         print(dataset.meta)
        #         print(dataset.profile)
        #         print(dataset.read_crs())
        # Loop through subdatasets (bands)
        for band in dataset.subdatasets:
            band_name = band.split(os.sep)[-1].split("/")[-1]
            # print(f"Band name: {band.split(os.sep)[-1].split('/')[-1]}")
            print(f"Band name: {band_name}")
            if band_name == required_bands[0]:
                with rio.open(band) as src:
                    dnb_at_sensor_radiance = src.read(1)
                    dnb_at_sensor_radiance_metadata = src.meta
                    dnb_at_sensor_radiance_profile = src.profile
#             qf_cloud_mask
#             qf_dnb_
#             utc_time

#             if band_name in required_bands:
#                 print(f"Band name: {band_name}")
#                 print()
#             print(band.meta)
# Extract DNB_At_Sensor_Radiance_500m, QF_Cloud_Mask, and QF_DNB